# MFI Assignment 10: Distance To Default Calculation

#### Importing Libraries

In [30]:
import pandas as pd
import warnings
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from scipy.optimize import fsolve
import scipy.stats as stat
from plotly.subplots import make_subplots


#### Importing FRED Datasets
BAFFM dataset, NBER Recession Data, CFSI Stress Index and Risk Free Rate from DTB3. All supplementary datasets are imported here first and then used subsequently when needed

In [81]:
## Reading BAAFFM Data
warnings.filterwarnings("ignore")

path = "D:/Data/"
fred_baaffm = pd.read_csv(path+'BAAFFM.csv', header = 0)

fred_baaffm['DATE'] = pd.to_datetime(fred_baaffm['DATE'])
fred_baaffm['DATE'] = pd.DatetimeIndex(fred_baaffm['DATE']).year
fred_baaffm = (fred_baaffm.groupby(['DATE'], as_index=False).mean().groupby('DATE')['BAAFFM'].mean())
fred_baaffm = fred_baaffm.reset_index()
fred_baaffm = fred_baaffm.rename(columns= {"DATE":"fyear"})
fred_baaffm.head()

,fyear,BAAFFM
0,1954,2.463333
1,1955,1.745000
2,1956,1.149167
3,1957,1.609167
4,1958,3.154167


In [84]:
## Readinog CFSI data

fred_cfsi = pd.read_csv(path+'CFSI.csv', header = 0)

fred_cfsi['DATE'] = pd.to_datetime(fred_cfsi['DATE'])
fred_cfsi['DATE'] = pd.DatetimeIndex(fred_cfsi['DATE']).year
fred_cfsi = (fred_cfsi.groupby(['DATE'], as_index=False).mean().groupby('DATE')['CFSI'].mean())
fred_cfsi = fred_cfsi.reset_index()
fred_cfsi = fred_cfsi.rename(columns= {"DATE":"fyear"})
fred_cfsi.head()

,fyear,CFSI
0,2011,1.256971
1,2012,0.960191
2,2013,-0.649726
3,2014,-0.989452
4,2015,0.173452


In [20]:
##Reading the interest rate data

rf_daily = pd.read_csv(path+"DTB3.csv")

rf_daily.columns = rf_daily.columns.str.strip().str.lower()
rf_daily["date"] = pd.to_datetime(rf_daily["date"])
rf_daily["dtb3"] = pd.to_numeric(rf_daily["dtb3"], errors='coerce')
rf_daily = rf_daily.dropna(subset = ["date","dtb3"]).reset_index(drop = True)
rf_daily["fyear"] = rf_daily["date"].dt.year
rf_daily["r"] = np.log(1+rf_daily["dtb3"]/100.0) 
rf_yearly = rf_daily.groupby(by = "fyear")["r"].first()
rf_yearly.head()


fyear
1970    0.076220
1971    0.047742
1972    0.036235
1973    0.050883
1974    0.072507
Name: r, dtype: float64

In [12]:
## NBER Recession Data 
nber_data =  pd.read_csv(path+'USREC.csv', header = 0)

nber_data['DATE'] = pd.to_datetime(nber_data['DATE'])
nber_data['fyear'] = pd.DatetimeIndex(nber_data['DATE']).year
nber_data = nber_data.iloc[1:]
nber_approx = nber_data.groupby('fyear').mean().round()
nber_approx.head()

#### Reading and Tidying up COMPUSTAT Data

In [9]:
cols = ["cusip","fyear","dlc","dltt","indfmt","datafmt","popsrc","fic","consol"]
funda_data = pd.read_csv(path+"funda.csv")
funda_data = funda_data[cols]

funda_data_filtered = funda_data[(funda_data["indfmt"]=="INDL")&
                                 (funda_data["datafmt"]=="STD")&
                                 (funda_data["popsrc"]=="D")&
                                 (funda_data["fic"]=="USA")&
                                 (funda_data["consol"]=="C")&
                                 (funda_data["fyear"]>=1970)&
                                 (funda_data["fyear"]<=2020)]
funda_data_filtered['cusip'] = funda_data_filtered['cusip'].str[0:6]
funda_data_filtered["date"] =  pd.to_datetime(funda_data_filtered['fyear'],format = "%Y")
funda_data_filtered['fyear'] = funda_data_filtered["date"].dt.year
funda_data_filtered['dlc'] = funda_data_filtered['dlc']*1000000
funda_data_filtered['dltt'] = funda_data_filtered['dltt']*1000000
funda_data_filtered = funda_data_filtered.sort_values("fyear").reset_index(drop = True)
funda_data_filtered["f"] = funda_data_filtered["dlc"] + funda_data_filtered["dltt"]*0.5
funda_data_filtered["f_lagged"] = funda_data_filtered.groupby(by = ["cusip"])["f"].shift()
del funda_data


In [11]:
funda_data_filtered.head()

,cusip,fyear,dlc,dltt,indfmt,datafmt,popsrc,fic,consol,date,f,f_lagged
0,000032,1970,12378000.0,917000.0,INDL,STD,D,USA,C,1970-01-01,12836500.0,NaN
1,108055,1970,289000.0,464000.0,INDL,STD,D,USA,C,1970-01-01,521000.0,NaN
2,597715,1970,5609000.0,46870000.0,INDL,STD,D,USA,C,1970-01-01,29044000.0,NaN
3,771010,1970,635000.0,1042000.0,INDL,STD,D,USA,C,1970-01-01,1156000.0,NaN
4,461097,1970,1219000.0,71878000.0,INDL,STD,D,USA,C,1970-01-01,37158000.0,NaN


#### Reading Daily CSRP Data
CSRP data is loaded once, then converted to yearly to use it with COMPUSTAT. 
However, we will need more granular data in Method 3, where we will use the data at monthly level, so I have ran this cell once and saved the monthly data as csv to avoid multiple processing. That's why the last 3 lines of the cell are commented. We will make use of the monthly data later in method 3

In [15]:
##Reading the CRSP daily data
## The following code is run once and then  saved monthly data to use in 3rd part
daily_data = pd.read_csv(path+"dsf_new.csv",usecols = ["DATE","CUSIP","PRC","RET","SHROUT"],dtype = {"CUSIP":"object"},nrows=10000000)
daily_data.columns = daily_data.columns.str.strip().str.lower()
daily_data = daily_data[daily_data["date"]>=19700101]
daily_data["date"] = pd.to_datetime(daily_data["date"],format = "%Y%m%d")
daily_data["fyear"] = daily_data["date"].dt.year
daily_data = daily_data.dropna(subset=["prc"])
daily_data = daily_data.dropna(subset=["ret"])
daily_data["prc"] = abs(daily_data["prc"])
daily_data["cusip"] = daily_data['cusip'].str[0:6]
daily_data["equity_value"] = abs(daily_data["prc"]*daily_data["shrout"])
daily_data = daily_data.sort_values("date").reset_index(drop = True)
daily_data["ret"] = pd.to_numeric(daily_data["ret"], errors='coerce')
#monthly_data = daily_data.groupby("cusip").resample('M', on='date').first().reset_index(drop =True)
#monthly_data.to_csv(path+"monthly_data.csv")
#del monthly_data

In [16]:
daily_data.head()

,date,cusip,prc,ret,shrout,fyear,equity_value
0,1970-01-02,556139,34.125,-0.052083,9327.0,1970,318283.875
1,1970-01-02,843673,46.750,0.008086,6809.0,1970,318320.750
2,1970-01-02,386532,25.375,0.004950,6433.0,1970,163237.375
3,1970-01-02,081689,33.750,0.015038,12493.0,1970,421638.750
4,1970-01-02,78375P,27.750,0.027778,18249.0,1970,506409.750


#### Calculate Annual Returns

In [17]:
annual_returns = pd.DataFrame(daily_data.groupby(by=['cusip','fyear']).apply(lambda x:
                                                       np.exp(np.sum(np.log(1+x['ret'])))))
annual_returns = annual_returns.rename(columns = {0:"cum_return"})
annual_returns = pd.DataFrame(annual_returns.groupby(by=['cusip'])["cum_return"].shift())
annual_returns.head()

cum_return
cusip  fyear            
000165 1983          NaN
       1984     1.142857
       1985     0.508332
       1986     2.655739
000209 1987          NaN

#### Preparing Quantities to Run Methods 1 and 2
Calcuating Volatility of equity value, equity value,  merging COMPUSTAT and CSRP

In [21]:
sigma_e = pd.DataFrame(daily_data.groupby(by=['cusip','fyear'])['ret'].std()*np.sqrt(250))
sigma_e = sigma_e.rename(columns = {"ret":"sigma_e"})
sigma_e =  pd.DataFrame(sigma_e.groupby(by = ["cusip"])["sigma_e"].shift())
sigma_e.head()
equity_value = pd.DataFrame(daily_data.groupby(by=['cusip','fyear'])['equity_value'].first())
final = pd.merge(annual_returns,sigma_e, on = ["cusip","fyear"],how = "inner")
final = final.merge(funda_data_filtered[["cusip","fyear","f_lagged"]],on = ["cusip","fyear"],how = "inner")
final.head()
final = final.merge(rf_yearly,on = "fyear",how = "inner")
final = final.merge(equity_value,on = ["cusip","fyear"],how = "inner")
final.sort_values("fyear").reset_index(drop  = True, inplace=True)
final.tail()

,cusip,fyear,cum_return,sigma_e,f_lagged,r,equity_value
38283,84857L,1971,1.262320,0.205302,51941000.0,0.047742,98372.500
38284,859264,1971,0.967228,0.335493,18478000.0,0.047742,1462096.625
38285,860486,1971,0.876571,0.287926,0.0,0.047742,122160.000
38286,86764P,1971,1.180111,0.210535,302828000.0,0.047742,1409372.250
38287,909214,1971,0.697896,0.391353,452412500.0,0.047742,1994155.000


### Naive Method 1
Note: I am not sampling 1000 companies in method 1 and method 2 as it is not extremely computation intensive and I want to re use the calculated values in method 3 to initialize sigma_v, and to have an initial estimate of any random 250 set of companies, I safe side calculate all of them to avoid any problems 

In [106]:
## DD and PD Supporting Functions
def distance_to_default(df,col_name,v,sigmav):
    df[col_name] = (np.log(df[v]/df["f_lagged"])+(df["r"]-df[sigmav]**2/2))/df[sigmav]

##Method 1 for Distance to Default
def naive_method(df,intercept,beta,keepVariables):
    sigma_d = df["sigma_e"]*beta + intercept
    df["v"] = df["equity_value"] + df["f_lagged"]
    df["sigma_v"] = (df["equity_value"]*df["sigma_e"]/df["v"]) + (df["f_lagged"]*sigma_d/df["v"])
    distance_to_default(df,"dd_naive"+"_"+str(intercept)+"_"+str(beta),"v","sigma_v")
    if keepVariables:
        return
    else:
        del df["v"]
        del df["sigma_v"]
        return        

In [27]:
naive_method(final,0.05,0.25,False)
naive_method(final,0.05,0.5,False)
naive_method(final,0,0.25,True)

In [97]:
final.head()

,cusip,fyear,cum_return,sigma_e,f_lagged,r,equity_value,dd_naive_0.05_0.25,dd_naive_0.05_0.5,v,sigma_v,dd_naive_0_0.25,v_method_2,sigma_v_method_2,dd_method_2,pd_naive_0.05_0.25,pd_naive_0.05_0.5,pd_naive_0_0.25,pd_method_2
0,000781,1983,0.485713,0.408360,16950000.0,0.07622,1861.50,0.425721,0.173180,16951861.50,0.102124,0.696364,1.570257e+07,0.000531,-0.421166,0.335156,0.431255,0.243100,6.631832e-01
1,000874,1983,1.133339,0.404174,1498000.0,0.07622,2822.00,0.439699,0.183284,1500822.00,0.101613,0.717813,1.390885e+06,0.000826,2.456667,0.330077,0.427287,0.236436,7.011629e-03
2,001015,1983,0.951955,0.367341,4505000.0,0.07622,6307.75,0.474960,0.214980,4511307.75,0.092220,0.795560,4.180694e+06,0.000556,2.713991,0.317408,0.414892,0.213144,3.323899e-03
3,001030,1983,1.510999,0.397504,6924000.0,0.07622,29138.00,0.459439,0.197796,6953138.00,0.100625,0.748883,6.444979e+06,0.001808,2.503069,0.322959,0.421603,0.226964,6.156084e-03
4,001038,1983,2.786490,0.196910,199500.0,0.07622,11338.25,1.206339,0.794961,210838.25,0.057169,2.271547,1.961974e+05,0.011379,5.225431,0.113843,0.213318,0.011557,8.687493e-08


### Equation Solver Method 2

In [87]:
#Required Equations To SOlve for Method 2
def equations(variables,*constants) :
    (v,sigma_v) = variables
    (firm_value,r,e, sigma_e) = constants
    d1 = (np.log(v/firm_value) +(r+0.5*sigma_v**2))/sigma_v 
    d2 = d1-sigma_v
    eq1 = e - v*stat.norm.cdf(d1, 0.0, 1.0) + np.exp(-r)*firm_value*stat.norm.cdf(d2, 0.0, 1.0)
    eq2 = sigma_e - (v*stat.norm.cdf(d1, 0.0, 1.0)*sigma_v)/e
    return (eq1, eq2)

## Method 2 
def equation_solver_method(data):
    data["v_method_2"] = 0
    data["sigma_v_method_2"] = 0
    data["dd_method_2"] = 0
    for i in range(len(data)):
        if(df.loc[i,:].isnull().values.any()):
            continue
        f_lagged = data.loc[i,"f_lagged"]
        r = data.loc[i,"r"]
        e = data.loc[i,"equity_value"]
        sigmae = data.loc[i,"sigma_e"]
        initial = (f_lagged,r, e,sigmae)
        data.loc[i,["v_method_2","sigma_v_method_2"]] = fsolve(equations,((e+f_lagged),0.10),args = initial)
    distance_to_default(data,"dd_method_2","v_method_2","sigma_v_method_2")

final.reset_index(drop  = True, inplace=True)
equation_solver_method(final)
final.head()

,cusip,fyear,cum_return,sigma_e,f_lagged,r,equity_value,dd_naive_0.05_0.25,dd_naive_0.05_0.5,v,sigma_v,dd_naive_0_0.25,v_method_2,sigma_v_method_2,dd_method_2,pd_naive_0.05_0.25,pd_naive_0.05_0.5,pd_naive_0_0.25,pd_method_2
0,000781,1983,0.485713,0.408360,16950000.0,0.07622,1861.50,0.425721,0.173180,16951861.50,0.102124,0.696364,1.570257e+07,0.000531,-0.421166,0.335156,0.431255,0.243100,6.631832e-01
1,000874,1983,1.133339,0.404174,1498000.0,0.07622,2822.00,0.439699,0.183284,1500822.00,0.101613,0.717813,1.390885e+06,0.000826,2.456667,0.330077,0.427287,0.236436,7.011629e-03
2,001015,1983,0.951955,0.367341,4505000.0,0.07622,6307.75,0.474960,0.214980,4511307.75,0.092220,0.795560,4.180694e+06,0.000556,2.713991,0.317408,0.414892,0.213144,3.323899e-03
3,001030,1983,1.510999,0.397504,6924000.0,0.07622,29138.00,0.459439,0.197796,6953138.00,0.100625,0.748883,6.444979e+06,0.001808,2.503069,0.322959,0.421603,0.226964,6.156084e-03
4,001038,1983,2.786490,0.196910,199500.0,0.07622,11338.25,1.206339,0.794961,210838.25,0.057169,2.271547,1.961974e+05,0.011379,5.225431,0.113843,0.213318,0.011557,8.687493e-08


### KMV Model: Method 3

In [201]:
def equation_method_3(variables,*args) :
    (v) = variables
    (f_lagged,r,e,sigmav) = args
    d1 = (np.log(v/f_lagged) +(r+0.5*sigmav**2))/(sigmav*np) 
    d2 = d1-sigmav
    eq1 = e - v*si.norm.cdf(d1, 0.0, 1.0) + np.exp(-r)*f_lagged*si.norm.cdf(d2, 0.0, 1.0)
    return (eq1)

def KMV_model_method(data, monthly_data):
    
    data["v_iterate"] = 0
    data["sigmav_iterate"] = 0
    i = 0
    while (i <len(data)):
        if(data.loc[i,:].isnull().values.any()):
            i += 1
            continue
        start_year = data.loc[i,"fyear"]
        cusip_id = data.loc[i,"cusip"]
        r = data.loc[i,"r"]
        e = data.loc[i,"equity_value"]
        error = 1000000
        time_steps = len(data[(data["fyear"]==start_year)
                              &(data["cusip"]==cusip_id)])
        
        ## Bring out the firm vinitial variables from the monthly data that we created
        f_lagged = monthly_data.loc[(monthly_data["fyear"]==(start_year+1))&
                                  (monthly_data["cusip"]==cusip_id),
                                  "f_lagged"].iloc[0]
        sigmav = data.loc[(data["fyear"]==(start_year+1))
                          &(data["cusip"]==cusip_id),"sigmav_solver"].iloc[0]
        
        if(sigmav == 0 or np.isnan(sigmav) or f_lagged== 0 or np.isnan(f_lagged)):
            ## We could not find information about this firm, so skip this firm from calculation
            i += time_steps
            continue
            
        monthly_values = monthly_data.loc[(monthly_data["fyear"]==(start_year))
                               &(monthly_data["cusip"]==cusip_id)].reset_index(drop = True)
        while(error > 0.01):
            value_time_series = []
            for j in range(time_steps):
                e = monthly_values.loc[j,"equity_value"]
                initial = (f_lagged,r,e,sigmav,1)
                value_time_series.append(fsolve(equation_method_3,(e+f_lagged),args = initial)[0])
            sigmav_n = (np.std(pd.DataFrame(value_time_series).pct_change())*np.sqrt(12)).iloc[0]
            error = abs(sigmav_n - sigmav)
            sigmav = sigmav_n
        initial = (f_lagged,r, e,sigmav,1)
        data.loc[(i+time_steps),["v_mthod_3","sigmav_method_3"]] = (fsolve(equation_v,(e+f_lagged),args = initial),sigmav_n)
        i += time_steps
    

#### Reading back monthly data

In [202]:
monthly_data = pd.read_csv(path+"monthly_data.csv")
monthly_data.head()

,Unnamed: 0,date,cusip,prc,ret,shrout,fyear,equity_value
0,0,20-08-1990,21,-3.3750,-0.055556,6369.0,1990.0,21495375.0
1,1,04-09-1990,21,-3.0625,-0.020000,6369.0,1990.0,19505062.5
2,2,01-10-1990,21,-2.8750,0.000000,6369.0,1990.0,18310875.0
3,3,01-11-1990,21,-2.8125,-0.010989,6369.0,1990.0,17912812.5
4,4,03-12-1990,21,-2.6250,0.000000,6369.0,1990.0,16718625.0


#### Sample and Prepare Data for Method 3
Note: We will use stored monthly data(to take firm value on each time series), all results from method 1 or 2 (to initialize) and sample 250 firms every year to carry out the calculations

In [203]:
monthly_data = monthly_data.merge(funda_data_filtered[["cusip","fyear","f_lagged"]],on = ["cusip","fyear"],how = "left")
cols_taken = ["cusip","fyear","cum_return","sigma_e","r","dd_naive_0.05_0.25"
              ,"dd_naive_0.05_0.5","dd_naive_0_0.25","pd_naive_0.05_0.25","pd_naive_0.05_0.5",
              "pd_naive_0_0.25","dd_method_2","dd_method_2","sigma_v_method_2"]
method_3_data = pd.merge(monthly_data[["date","fyear","cusip","equity_value","f_lagged"]],final[cols_taken],on = ["cusip","fyear"],how = "inner")
method_3_data.sort_values("date").reset_index(drop  = True, inplace=True)
method_3_data = method_3_data[method_3_data.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)] 
companies_to_sample_from = pd.DataFrame(final.groupby("fyear")["cusip"].unique()).reset_index()
sample_250 = pd.DataFrame()

for i in range(len(companies_to_sample_from)):
    df1 = method_3_data[method_3_data["cusip"].isin(np.random.choice(k.loc[i,"cusip"],250)) & (method_3_data["fyear"]==k.loc[i,"fyear"])]
    sample_250 = sample_250.append(df1)
sample_250 = sample_250.loc[:,sample_250.columns!="f_lagged"].reset_index(drop = True)
monthly_data = monthly_data.dropna().reset_index(drop = True)
monthly_data = pd.merge(sample_250[sub_sample_1.columns[~sample_250.columns.isin(["fyear","equity_value","f_lagged"])]],monthly_data[["date","fyear","cusip","equity_value","f_lagged"]],on = ["date","cusip"],how= "right")
sample_250.head()

,date,fyear,cusip,equity_value,cum_return,sigma_e,r,dd_naive_0.05_0.25,dd_naive_0.05_0.5,dd_naive_0_0.25,pd_naive_0.05_0.25,pd_naive_0.05_0.5,pd_naive_0_0.25,dd_method_2,dd_method_2,sigma_v_method_2
0,04-01-1971,1971.0,02209S,1.108135e+09,1.422493,0.271185,0.047742,0.369715,0.179941,0.707195,0.355797,0.428599,0.239723,1.442869,1.442869,0.002108
1,01-02-1971,1971.0,02209S,1.209902e+09,1.422493,0.271185,0.047742,0.369715,0.179941,0.707195,0.355797,0.428599,0.239723,1.442869,1.442869,0.002108
2,01-03-1971,1971.0,02209S,1.215556e+09,1.422493,0.271185,0.047742,0.369715,0.179941,0.707195,0.355797,0.428599,0.239723,1.442869,1.442869,0.002108
3,01-04-1971,1971.0,02209S,1.356900e+09,1.422493,0.271185,0.047742,0.369715,0.179941,0.707195,0.355797,0.428599,0.239723,1.442869,1.442869,0.002108
4,03-05-1971,1971.0,02209S,1.501071e+09,1.422493,0.271185,0.047742,0.369715,0.179941,0.707195,0.355797,0.428599,0.239723,1.442869,1.442869,0.002108


In [204]:
KMV_model_method(sample_250,monthly_data) 
sample_250 = sample_250[sample_250.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)] 
method_3_final = sample_250.merge(monthly_data[["cusip","date","f_lagged"]],on = ["cusip","date"],how = "left")
distance_default(method_3_final,"dd_method_3","v_method_3","sigmav_method_3")
probability_default(method_3_final,"pd_method_3","dd_iterate")
cols_method_3 = ["date","fyear","cusip","dd_method_3","pd_method_3","v_method_3","sigmav_method_3"]
method_3 = method_3[cols_method_3]
method_3_final.head(5)

,date,fyear,cusip,dd_method_3,pd_method_3,v_method_3,sigmav_method_3
0,1987-01-02,1987.0,000872,1.869065,0.030807,6.112028e+06,0.955322
1,2014-01-02,2014.0,00123Q,2.043682,0.020492,6.941002e+10,0.056337
2,2020-01-02,2020.0,00164V,4.032863,0.000028,3.327452e+09,0.164601
3,2002-01-02,2002.0,004308,4.358460,0.000007,1.199040e+08,0.211512
4,2002-01-02,2002.0,005094,-0.400551,0.655624,4.291978e+08,1.535766


In [205]:
method_3_cols = ["date","fyear","cusip","dd_iterate","pd_iterate","v_iterate","sigmav_iterate"]
method_3_final = pd.read_csv(path+"file.csv")
method_3_final = method_3_final[method_3_cols]
method_3_final = method_3_final.rename(columns = {"pd_iterate":"pd_method_3","dd_iterate":"dd_method_3","v_iterate":"v_method_3","sigmav_iterate":"sigmav_method_3"})
method_3_final.head()
#final_method_3 = final_method_3.merge(final_data[["cusip","date","f_lagged"]],on = ["cusip","date"],how = "left")

,date,fyear,cusip,dd_method_3,pd_method_3,v_method_3,sigmav_method_3
0,1987-01-02,1987.0,000872,1.869065,0.030807,6.112028e+06,0.955322
1,2014-01-02,2014.0,00123Q,2.043682,0.020492,6.941002e+10,0.056337
2,2020-01-02,2020.0,00164V,4.032863,0.000028,3.327452e+09,0.164601
3,2002-01-02,2002.0,004308,4.358460,0.000007,1.199040e+08,0.211512
4,2002-01-02,2002.0,005094,-0.400551,0.655624,4.291978e+08,1.535766


In [206]:
all_metrics = pd.merge(method_3_final, final, on = ["fyear","cusip"])

### Calculate Probability of Default for all calculated DDs

In [64]:
final["pd_naive_0.05_0.25"] = stat.norm.cdf(-final["dd_naive_0.05_0.25"], 0.0, 1.0)
final["pd_naive_0.05_0.5"] = stat.norm.cdf(-final["dd_naive_0.05_0.5"], 0.0, 1.0)
final["pd_naive_0_0.25"] = stat.norm.cdf(-final["dd_naive_0_0.25"], 0.0, 1.0)
final["pd_method_2"] = stat.norm.cdf(-final["dd_method_2"], 0.0, 1.0)
final.head()

,cusip,fyear,cum_return,sigma_e,f_lagged,r,equity_value,dd_naive_0.05_0.25,dd_naive_0.05_0.5,v,sigma_v,dd_naive_0_0.25,v_method_2,sigma_v_method_2,dd_method_2,pd_naive_0.05_0.25,pd_naive_0.05_0.5,pd_naive_0_0.25,pd_method_2
3,000781,1983,0.485713,0.408360,16950000.0,0.07622,1861.50,0.425721,0.173180,16951861.50,0.102124,0.696364,1.570257e+07,0.000531,-0.421166,0.335156,0.431255,0.243100,6.631832e-01
5,000874,1983,1.133339,0.404174,1498000.0,0.07622,2822.00,0.439699,0.183284,1500822.00,0.101613,0.717813,1.390885e+06,0.000826,2.456667,0.330077,0.427287,0.236436,7.011629e-03
6,001015,1983,0.951955,0.367341,4505000.0,0.07622,6307.75,0.474960,0.214980,4511307.75,0.092220,0.795560,4.180694e+06,0.000556,2.713991,0.317408,0.414892,0.213144,3.323899e-03
7,001030,1983,1.510999,0.397504,6924000.0,0.07622,29138.00,0.459439,0.197796,6953138.00,0.100625,0.748883,6.444979e+06,0.001808,2.503069,0.322959,0.421603,0.226964,6.156084e-03
8,001038,1983,2.786490,0.196910,199500.0,0.07622,11338.25,1.206339,0.794961,210838.25,0.057169,2.271547,1.961974e+05,0.011379,5.225431,0.113843,0.213318,0.011557,8.687493e-08


#### Supporting Generalized Plotting Functions
These are directly taken from our the momentum assignment submission from Finance and Investments class

In [79]:
def plot_chart(df,title_name,y1_name,date_column):
    x = df["fyear"].values
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    for i in range(len(df.columns)-1):
        fig.add_trace(go.Scatter(name=df.columns[i+1],x=x, y=df.iloc[:,i+1]),secondary_y=False)
    fig.update_yaxes(title_text=""+str(y1_name), secondary_y=False)   
    fig.update_layout(barmode='relative',showlegend = True, title_text="Title:"+title_name)
    fig.show()
    
def plot_chart_with_macro(df,title_name,y1_name,date_column):
    x = df["fyear"].values
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    for i in range(len(df.columns)-2):
        fig.add_trace(go.Scatter(name=df.columns[i+1],x=x, y=df.iloc[:,i+1]),secondary_y=False)
    fig.update_yaxes(title_text=""+str(y1_name), secondary_y=False)
    fig.add_trace(go.Scatter(name=df.columns[-1],x=x, y=df.iloc[:,-1]), secondary_y=True)
    fig.update_yaxes(title_text="Economic Indicator in %", secondary_y=True)      
    fig.update_layout(barmode='relative',showlegend = True, title_text="Title:"+title_name)
    fig.show()

### Descriptive Stats - Both PD,DD for first 2 methods
Note: Most of the plots and descriptive stats comparisons we will divide in 2 parts: All firms for method1 and 2, sampled 250 firms for all 3 methods 

In [55]:
all_pd_results = ["pd_naive_0.05_0.25","pd_naive_0.05_0.5","pd_naive_0_0.25","pd_method_2"]
all_dd_results = ["dd_naive_0.05_0.25","dd_naive_0.05_0.5","dd_naive_0_0.25","dd_method_2"]
all_results = ["dd_naive_0.05_0.25","dd_naive_0.05_0.5","dd_naive_0_0.25","pd_naive_0.05_0.25","pd_naive_0.05_0.5",
              "pd_naive_0_0.25","dd_method_2","pd_method_2"]

final = final[final.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)] 
descriptive_stats_all = final[all_results].describe()
descriptive_stats_all

,dd_naive_0.05_0.25,dd_naive_0.05_0.5,dd_naive_0_0.25,pd_naive_0.05_0.25,pd_naive_0.05_0.5,pd_naive_0_0.25,dd_method_2,pd_method_2
count,19117.000000,19117.000000,19117.000000,1.911700e+04,1.911700e+04,19117.000000,19117.000000,19117.000000
mean,0.296359,0.103027,0.496746,4.065582e-01,4.760141e-01,0.351377,0.795708,0.220248
std,0.610558,0.575487,1.041700,1.193059e-01,1.146522e-01,0.144493,14.066862,0.292871
min,-2.091015,-4.172309,-2.065872,5.040511e-56,6.276758e-54,0.000000,-1169.038577,0.000000
25%,0.017897,-0.097752,0.085516,3.551141e-01,4.388981e-01,0.266782,0.372311,0.003945
50%,0.167460,0.001423,0.298310,4.335040e-01,4.994323e-01,0.382733,1.472137,0.070492
75%,0.371550,0.153764,0.622575,4.928603e-01,5.389353e-01,0.465925,2.656703,0.354831
max,15.725713,15.417144,97.426810,9.817366e-01,9.999849e-01,0.980580,395.918008,1.000000


In [169]:
all_results_250 = all_results + ["dd_method_3","pd_method_3"]
descriptive_stats_250 = all_metrics[all_results_250].describe()
descriptive_stats_250

,dd_naive_0.05_0.25,dd_naive_0.05_0.5,dd_naive_0_0.25,pd_naive_0.05_0.25,pd_naive_0.05_0.5,pd_naive_0_0.25,dd_method_2,pd_method_2,dd_method_3,pd_method_3
count,75.000000,75.000000,75.000000,7.500000e+01,7.500000e+01,7.500000e+01,75.000000,7.500000e+01,75.000000,7.500000e+01
mean,0.220401,0.080022,0.382226,4.355391e-01,4.877580e-01,3.810256e-01,2.519319,1.344826e-01,8.485280,8.411022e-03
std,0.608239,0.591691,0.656810,1.045895e-01,9.330314e-02,1.271863e-01,3.364662,2.659072e-01,4.534682,4.395942e-02
min,-0.224523,-0.549658,-0.192005,4.012989e-07,5.818710e-07,3.524167e-07,-11.801363,2.829750e-15,0.377109,1.744215e-100
25%,0.000923,-0.058974,0.077877,4.315294e-01,4.770671e-01,3.483099e-01,1.176165,1.565619e-06,5.438588,1.212835e-26
50%,0.087332,0.005618,0.227178,4.652037e-01,4.977587e-01,4.101428e-01,3.131010,8.710299e-04,8.268415,6.787609e-17
75%,0.172506,0.057516,0.389992,4.996319e-01,5.235136e-01,4.689629e-01,4.668752,1.210451e-01,10.619225,3.395650e-08
max,4.934735,4.861713,4.960027,5.888247e-01,7.087229e-01,5.761308e-01,7.811314,1.000000e+00,21.247345,3.530464e-01


### Descriptive Stats Each Year (Both PD,DD for 2 methods)

In [65]:
descriptive_stats_across_time = final[["fyear"]+all_results].groupby("fyear").describe()
descriptive_stats_across_time.columns = descriptive_stats_across_time.columns.map('{0[0]}_{0[1]}'.format)
descriptive_stats_across_time = descriptive_stats_across_time.reset_index()
descriptive_stats_across_time.tail(10)

,fyear,dd_naive_0.05_0.25_count,dd_naive_0.05_0.25_mean,dd_naive_0.05_0.25_std,dd_naive_0.05_0.25_min,dd_naive_0.05_0.25_25%,dd_naive_0.05_0.25_50%,dd_naive_0.05_0.25_75%,dd_naive_0.05_0.25_max,dd_naive_0.05_0.5_count,...,dd_method_2_75%,dd_method_2_max,pd_method_2_count,pd_method_2_mean,pd_method_2_std,pd_method_2_min,pd_method_2_25%,pd_method_2_50%,pd_method_2_75%,pd_method_2_max
40,2011,240.0,0.113531,0.517695,-0.198325,-0.051425,-0.013735,0.077012,5.380599,240.0,...,2.320974,9.293560,240.0,0.370386,0.400793,7.460591e-21,0.010145,0.178197,0.736055,1.0
41,2012,337.0,0.136919,0.809678,-0.242440,-0.070442,-0.044968,0.024391,10.334314,337.0,...,1.923572,11.769535,337.0,0.346460,0.346103,2.801614e-32,0.027204,0.266457,0.536130,1.0
42,2013,445.0,0.153109,0.722260,-0.545031,-0.047788,-0.019713,0.049154,8.036456,445.0,...,2.457304,27.065443,445.0,0.360259,0.394936,1.256765e-161,0.006999,0.192574,0.717812,1.0
43,2014,624.0,0.173846,0.735511,-0.910587,-0.041789,-0.009783,0.063505,11.556776,624.0,...,2.473544,29.414189,624.0,0.377463,0.401006,1.808233e-190,0.006689,0.217016,0.838110,1.0
44,2015,709.0,0.158086,0.788941,-0.765232,-0.052471,-0.025200,0.033050,10.869925,709.0,...,2.436242,21.351114,709.0,0.349291,0.382240,1.903777e-101,0.007420,0.168700,0.593069,1.0
45,2016,671.0,0.086423,0.565923,-1.706348,-0.050277,-0.019030,0.023084,5.975354,671.0,...,2.317129,18.013288,671.0,0.341643,0.355882,7.662445e-73,0.010249,0.240880,0.546293,1.0
46,2017,615.0,0.115048,0.709227,-0.503426,-0.036448,0.004964,0.046003,8.291220,615.0,...,2.393362,9.881615,615.0,0.307716,0.351254,2.501004e-23,0.008347,0.165403,0.492694,1.0
47,2018,574.0,0.268095,1.064354,-0.411284,0.042693,0.097219,0.159661,15.725713,574.0,...,3.540563,16.417974,574.0,0.235553,0.344742,7.111842e-61,0.000200,0.029142,0.363326,1.0
48,2019,549.0,0.258673,0.611606,-0.847152,0.081032,0.159162,0.211225,8.184791,549.0,...,3.655054,9.179056,549.0,0.128844,0.243908,2.174427e-20,0.000129,0.007876,0.124713,1.0
49,2020,590.0,0.193230,0.622358,-0.682902,0.025411,0.099142,0.160161,7.803129,590.0,...,3.309975,72.797891,590.0,0.217547,0.321022,0.000000e+00,0.000467,0.033259,0.330682,1.0


### Descriptive Stats Each Year (Both PD,DD for 3 methods, 250 firms)

In [170]:
descriptive_stats_across_time_250 = all_metrics[["fyear"]+all_results_250].groupby("fyear").describe()
descriptive_stats_across_time_250.columns = descriptive_stats_across_time_250.columns.map('{0[0]}_{0[1]}'.format)
descriptive_stats_across_time_250 = descriptive_stats_across_time_250.reset_index()
descriptive_stats_across_time_250.tail(10)

,fyear,dd_naive_0.05_0.25_count,dd_naive_0.05_0.25_mean,dd_naive_0.05_0.25_std,dd_naive_0.05_0.25_min,dd_naive_0.05_0.25_25%,dd_naive_0.05_0.25_50%,dd_naive_0.05_0.25_75%,dd_naive_0.05_0.25_max,dd_naive_0.05_0.5_count,...,dd_method_3_75%,dd_method_3_max,pd_method_3_count,pd_method_3_mean,pd_method_3_std,pd_method_3_min,pd_method_3_25%,pd_method_3_50%,pd_method_3_75%,pd_method_3_max
19,2008.0,1.0,0.234313,NaN,0.234313,0.234313,0.234313,0.234313,0.234313,1.0,...,8.002705,8.002705,1.0,6.085780e-16,NaN,6.085780e-16,6.085780e-16,6.085780e-16,6.085780e-16,6.085780e-16
20,2009.0,1.0,4.934735,NaN,4.934735,4.934735,4.934735,4.934735,4.934735,1.0,...,21.247345,21.247345,1.0,1.744215e-100,NaN,1.744215e-100,1.744215e-100,1.744215e-100,1.744215e-100,1.744215e-100
21,2013.0,2.0,0.039255,0.078806,-0.016469,0.011393,0.039255,0.067117,0.094979,2.0,...,8.493288,8.612047,2.0,1.030846e-16,1.407089e-16,3.588363e-18,5.333646e-17,1.030846e-16,1.528327e-16,2.025808e-16
22,2014.0,1.0,-0.113114,NaN,-0.113114,-0.113114,-0.113114,-0.113114,-0.113114,1.0,...,2.455816,2.455816,1.0,7.028265e-03,NaN,7.028265e-03,7.028265e-03,7.028265e-03,7.028265e-03,7.028265e-03
23,2015.0,1.0,-0.034277,NaN,-0.034277,-0.034277,-0.034277,-0.034277,-0.034277,1.0,...,12.225126,12.225126,1.0,1.141142e-34,NaN,1.141142e-34,1.141142e-34,1.141142e-34,1.141142e-34,1.141142e-34
24,2016.0,12.0,-0.010523,0.021029,-0.067198,-0.013286,0.000175,0.000923,0.000923,12.0,...,16.422705,16.422705,12.0,2.988250e-03,8.178583e-03,6.578438e-61,6.578438e-61,3.553076e-58,1.144897e-22,2.792958e-02
25,2017.0,3.0,0.023644,0.009867,0.013155,0.019095,0.025035,0.028889,0.032742,3.0,...,8.448709,9.192430,3.0,4.298095e-09,7.444512e-09,1.920331e-20,3.271394e-15,6.542770e-15,6.447142e-09,1.289428e-08
26,2018.0,3.0,0.070796,0.052864,0.032651,0.040623,0.048596,0.089868,0.131140,3.0,...,10.359521,10.888958,3.0,2.370132e-14,4.105188e-14,6.505996e-28,2.088770e-23,4.177475e-23,3.555197e-14,7.110395e-14
27,2019.0,3.0,0.085137,0.116026,-0.039862,0.033011,0.105883,0.147637,0.189391,3.0,...,6.110137,6.910958,3.0,3.299475e-03,5.714811e-03,2.406955e-12,2.751057e-08,5.501872e-08,4.949213e-03,9.898371e-03
28,2020.0,21.0,0.111771,0.039127,0.017384,0.086140,0.123615,0.134607,0.209151,21.0,...,10.058350,11.622757,21.0,2.964490e-05,1.296725e-04,1.578884e-31,4.220033e-24,6.787609e-17,4.300581e-09,5.949750e-04


### Correlations PD

In [46]:
correlation_table_PD = final[all_pd_results].corr()
correlation_table_PD

,pd_naive_0.05_0.25,pd_naive_0.05_0.5,pd_naive_0_0.25,pd_method_2
pd_naive_0.05_0.25,1.000000,0.969126,0.967208,0.407482
pd_naive_0.05_0.5,0.969126,1.000000,0.922613,0.400314
pd_naive_0_0.25,0.967208,0.922613,1.000000,0.418632
pd_method_2,0.407482,0.400314,0.418632,1.000000


### Correlations DD

In [48]:
correlation_table_DD = final[all_dd_results].corr()
correlation_table_DD

,dd_naive_0.05_0.25,dd_naive_0.05_0.5,dd_naive_0_0.25,dd_method_2
dd_naive_0.05_0.25,1.000000,0.988821,0.715913,0.090189
dd_naive_0.05_0.5,0.988821,1.000000,0.698910,0.079310
dd_naive_0_0.25,0.715913,0.698910,1.000000,0.141255
dd_method_2,0.090189,0.079310,0.141255,1.000000


### Correlations DD All 3

In [211]:
correlation_table_DD_250 = all_metrics[all_dd_results+["dd_method_3"]].corr()
correlation_table_DD_250

,dd_naive_0.05_0.25,dd_naive_0.05_0.5,dd_naive_0_0.25,dd_method_2,dd_method_3
dd_naive_0.05_0.25,1.000000,0.986234,0.977140,0.104263,0.348398
dd_naive_0.05_0.5,0.986234,1.000000,0.946201,0.145331,0.415777
dd_naive_0_0.25,0.977140,0.946201,1.000000,0.139482,0.350939
dd_method_2,0.104263,0.145331,0.139482,1.000000,0.406475
dd_method_3,0.348398,0.415777,0.350939,0.406475,1.000000


### Mean DD First 2 Methods vs BAAFM

In [218]:
mean_all_columns = ["fyear","dd_naive_0.05_0.25_mean", "dd_naive_0.05_0.5_mean", "dd_naive_0_0.25_mean","dd_method_2_mean"]
mean_all_methods = descriptive_stats_across_time[mean_all_columns]
plot_chart(mean_all_methods,"Distance to default 2 methods","DD in Years", "fyear")

mean_plot = pd.merge(mean_all_methods,fred_baaffm,on="fyear",how="inner")
plot_chart_with_macro(mean_plot,"Distance to default 2 methods vs BAAFFM","DD in Years", "fyear")

### Mean all 3 methods vs BAAFM

In [171]:
mean_all3_columns = ["fyear","dd_naive_0.05_0.25_mean", "dd_naive_0.05_0.5_mean", "dd_naive_0_0.25_mean","dd_method_2_mean","dd_method_3_mean","dd_method_3_mean"]
mean_all3_methods = descriptive_stats_across_time_250[mean_all3_columns]
plot_chart(mean_all3_methods,"Distance to default all methods","DD in Years", "fyear")

mean_plot_all3 = pd.merge(mean_all3_methods,fred_baaffm,on="fyear",how="inner")
plot_chart_with_macro(mean_plot_all3,"Distance to default all methods vs BAAFFM","DD in Years", "fyear")

### Mean PD 2 methods vs BAAFM

In [219]:
mean_pd_all_columns = ["fyear","pd_naive_0.05_0.25_mean","pd_naive_0.05_0.5_mean","pd_naive_0_0.25_mean","pd_method_2_mean"]
mean_pd_all_methods = descriptive_stats_across_time[mean_pd_all_columns]
plot_chart(mean_pd_all_methods,"2 Methods vs BAAFFM","Probability of Default", "fyear")

mean_pd_plot = pd.merge(mean_pd_all_methods,fred_baaffm,on="fyear",how="inner")
plot_chart_with_macro(mean_pd_plot,"2 Methods vs BAAFFM","Probability of Default", "fyear")

### Median DD 2 methods vs CF Stress Index

In [220]:
p50_all_columns = ["fyear","dd_naive_0.05_0.25_50%", "dd_naive_0.05_0.5_50%", "dd_naive_0_0.25_50%","dd_method_2_50%"]
p50_all_methods = descriptive_stats_across_time[p50_all_columns]

to_plot = pd.merge(p50_all_methods,fred_cfsi,on="fyear",how="inner")
plot_chart_with_macro(to_plot,"Results vs CF Stress Index","DD in Years", "fyear")

### Median PD all methods vs BAAFM

In [217]:
p50_250_columns = ["fyear","pd_naive_0.05_0.25_50%", "pd_naive_0.05_0.5_50%", "pd_naive_0_0.25_50%","pd_method_2_50%","pd_method_3_50%"]
p50_250_methods = descriptive_stats_across_time_250[p50_250_columns]

to_plot = pd.merge(p50_250_methods,fred_baaffm,on="fyear",how="inner")
plot_chart_with_macro(to_plot,"All methods vs BAAFFM","Probability of Default", "fyear")

### p25 DD 2 methods

In [222]:
p25_all_columns = ["fyear","dd_naive_0.05_0.25_25%", "dd_naive_0.05_0.5_25%", "dd_naive_0_0.25_25%","dd_method_2_25%"]
p25_all_methods = descriptive_stats_across_time[p25_all_columns]
plot_chart(p25_all_methods,"Naive Method vs BAAFFM","DD in Years", "fyear")

p25_plot = pd.merge(p25_all_methods,fred_baaffm,on="fyear",how="inner")
plot_chart_with_macro(p25_plot,"Naive Method vs BAAFFM","DD in Years", "fyear")

### p50 of all methods 

In [221]:
p50_all_columns = ["fyear","dd_naive_0.05_0.25_50%", "dd_naive_0.05_0.5_50%", "dd_naive_0_0.25_50%","dd_method_2_50%"]
p50_all_methods = descriptive_stats_across_time[p50_all_columns]
plot_chart(p50_all_methods,"Naive Method vs BAAFFM","DD in Years", "fyear")

to_plot = pd.merge(p50_all_methods,fred_baaffm,on="fyear",how="inner")
plot_chart_with_macro(to_plot,"Naive Method vs BAAFFM","DD in Years", "fyear")

### p75 of all methods 

In [223]:
p75_all_columns = ["fyear","dd_naive_0.05_0.25_75%", "dd_naive_0.05_0.5_75%", "dd_naive_0_0.25_75%","dd_method_2_75%"]
p75_all_methods = descriptive_stats_across_time[p75_all_columns]
plot_chart(p75_all_methods,"Naive Method vs BAAFFM","DD in Years", "fyear")

to_plot = pd.merge(p75_all_methods,fred_baaffm,on="fyear",how="inner")
plot_chart_with_macro(to_plot,"Naive Method vs BAAFFM","DD in Years", "fyear")

### Plotting DD wtih NBER Recession Data

In [225]:
mean_nber = pd.merge(p50_all_methods,nber_approx,on="fyear",how="inner")
plot_chart_with_macro(mean_nber,"Results vs NBER Recession Data","DD in Years", "fyear")

#### Descriptive Stats for Recession Years

In [75]:
non_recession_years = nber_approx[nber_approx['USREC']==0]
recession_years = nber_approx[nber_approx['USREC']==1]

final_recession_years = pd.merge(final[["fyear"]+all_results],recession_years,on="fyear",how="inner")
descriptive_stats_recession_years = final_recession_years.groupby("fyear").describe()
descriptive_stats_recession_years.columns = descriptive_stats_recession_years.columns.map('{0[0]}_{0[1]}'.format)
descriptive_stats_recession_years = descriptive_stats_recession_years.reset_index()
descriptive_stats_recession_years

,fyear,dd_naive_0.05_0.25_count,dd_naive_0.05_0.25_mean,dd_naive_0.05_0.25_std,dd_naive_0.05_0.25_min,dd_naive_0.05_0.25_25%,dd_naive_0.05_0.25_50%,dd_naive_0.05_0.25_75%,dd_naive_0.05_0.25_max,dd_naive_0.05_0.5_count,...,pd_method_2_75%,pd_method_2_max,USREC_count,USREC_mean,USREC_std,USREC_min,USREC_25%,USREC_50%,USREC_75%,USREC_max
0,1974,145.0,0.546656,0.498469,-0.036926,0.343909,0.476225,0.593753,3.816224,145.0,...,0.190988,0.800534,145.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,1982,161.0,0.871824,0.584186,-0.030187,0.557197,0.803678,0.998725,5.002421,161.0,...,0.459419,0.921969,161.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2,2001,349.0,0.235219,0.398349,-0.442222,0.078163,0.193085,0.298116,4.478097,349.0,...,0.527262,0.976519,349.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,2008,214.0,0.389198,0.852075,-0.145351,0.150874,0.202357,0.288944,9.628432,214.0,...,0.068134,1.000000,214.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


#### Descriptive Stats for Non Recession Years

In [77]:
final_non_recession_years = pd.merge(final[["fyear"]+all_results],non_recession_years,on="fyear",how="inner")
descriptive_stats_non_recession_years = final_non_recession_years.groupby("fyear").describe()
descriptive_stats_non_recession_years.columns = descriptive_stats_non_recession_years.columns.map('{0[0]}_{0[1]}'.format)
descriptive_stats_non_recession_years = descriptive_stats_non_recession_years.reset_index()
descriptive_stats_non_recession_years.tail(10)

,fyear,dd_naive_0.05_0.25_count,dd_naive_0.05_0.25_mean,dd_naive_0.05_0.25_std,dd_naive_0.05_0.25_min,dd_naive_0.05_0.25_25%,dd_naive_0.05_0.25_50%,dd_naive_0.05_0.25_75%,dd_naive_0.05_0.25_max,dd_naive_0.05_0.5_count,...,pd_method_2_75%,pd_method_2_max,USREC_count,USREC_mean,USREC_std,USREC_min,USREC_25%,USREC_50%,USREC_75%,USREC_max
36,2011,240.0,0.113531,0.517695,-0.198325,-0.051425,-0.013735,0.077012,5.380599,240.0,...,0.736055,1.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,2012,337.0,0.136919,0.809678,-0.242440,-0.070442,-0.044968,0.024391,10.334314,337.0,...,0.536130,1.0,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,2013,445.0,0.153109,0.722260,-0.545031,-0.047788,-0.019713,0.049154,8.036456,445.0,...,0.717812,1.0,445.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,2014,624.0,0.173846,0.735511,-0.910587,-0.041789,-0.009783,0.063505,11.556776,624.0,...,0.838110,1.0,624.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,2015,709.0,0.158086,0.788941,-0.765232,-0.052471,-0.025200,0.033050,10.869925,709.0,...,0.593069,1.0,709.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,2016,671.0,0.086423,0.565923,-1.706348,-0.050277,-0.019030,0.023084,5.975354,671.0,...,0.546293,1.0,671.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,2017,615.0,0.115048,0.709227,-0.503426,-0.036448,0.004964,0.046003,8.291220,615.0,...,0.492694,1.0,615.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,2018,574.0,0.268095,1.064354,-0.411284,0.042693,0.097219,0.159661,15.725713,574.0,...,0.363326,1.0,574.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,2019,549.0,0.258673,0.611606,-0.847152,0.081032,0.159162,0.211225,8.184791,549.0,...,0.124713,1.0,549.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,2020,590.0,0.193230,0.622358,-0.682902,0.025411,0.099142,0.160161,7.803129,590.0,...,0.330682,1.0,590.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
